In [1]:
import pandas as pd
import numpy as np

import datetime as dt

In [2]:
# abrir o arquivo da tape
filename = "NEG_BMF_20191126.TXT"
pathname = "../../data/b3/"
#f = open('../../data/b3/NEG_BMF_20200527.txt','r')
f = open(pathname + filename,'r')

In [3]:
# iterar o arquivo linha a linha, criar o index e salvar num arquivo com _index.

f.seek(45)
nlines = int(f.read(9))-2 # desconta cabeçalho e rodapé

f.seek(55)
data = []

ticker = ''
nstart = 0
nend = 0
for nline in range(nlines):
    line = f.readline()
    ticker_ = line[11:61]
    if ticker_ != ticker:
        data.append([ticker, nstart, nend])
        ticker = ticker_
        nstart = nline
    nend = nline
data.pop(0)

# O terceiro passo é pegar o dataframe de dados e jogar num .csv
# Cria um dataframe com os dados do index
idf = pd.DataFrame(data, columns=['Ticker', 'Linicial', 'Lfinal'])
# https://stackoverflow.com/questions/37351183/how-to-parse-all-the-values-in-a-column-of-a-dataframe
idf['Ticker'] = idf.Ticker.str.replace(' ','')
# Antes de transformar a coluna Ticker em Index do dataframe, precisamos dar um parse para remover espaços em branco
idf = idf.set_index('Ticker')
# https://stackoverflow.com/questions/22211737/how-to-sort-a-pandas-dataframe-by-index
idf.sort_index(inplace=True)
indexname = filename[:-4] + "_index" + filename[-4:]
idf.to_csv(pathname + indexname, header=None, sep=' ', mode='w')

In [12]:
# INPUTS

# Escolher código do papel
#ticker = 'DOLZ19'
ticker = 'WDOZ19'

# Escolher sampling frequency
freq = '1Min'

# Escolher output file name
#outfilename = "DOLZZ19_20191126_1Min.csv" # vai jogar na pasta pathname (no meu caso, "../../data/b3/")
outfilename = "WDOZ19_20191126_1Min.csv" # vai jogar na pasta pathname (no meu caso, "../../data/b3/")

In [13]:
# Abre o index do dia em questão
i = pd.read_csv(pathname + indexname, header = None, sep = ' ', index_col = 0, \
                names = ['Linicial', 'Lfinal'])

Linicial = i.loc[ticker].iloc[0]['Linicial']
Lfinal = i.loc[ticker].iloc[0]['Lfinal']

# Puxa os dados de cada linha e joga em "data"
f.seek(23)
dia = f.read(10)

start = 55
linesize = 238

data = []

for line_n in range(Linicial, Lfinal+1):
    f.seek(start+line_n*linesize)
    line = f.readline()
    # Preço, Quantidade, Time
    data.append([ \
        line[74:74+19], \
        line[94:94+18], \
        dia + ' ' + line[113:113+12]
    ])

# Cria um Dataframe bonitinho de transações negociadas
bneg = pd.DataFrame(data, columns=['Price', 'Quant', 'Timestamp'])
bneg['Price'] = bneg['Price'].astype(float)
bneg['Quant'] = bneg['Quant'].astype(int)
bneg['Timestamp'] = pd.to_datetime(bneg['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
bneg = bneg.set_index('Timestamp')

# Pega a frequencia de sampling desejada e cria um Dataframe OHLC + quantidade
ohlc = bneg['Price'].resample(freq).ohlc()
quant = bneg['Quant'].resample(freq).sum()
ohlc['Quant'] = quant

# joga num .csv no folder dos dados de entrada e com o nome escolhido nas INPUTS
ohlc.to_csv(pathname + outfilename, header=None, sep=';', mode='w')
ohlc

,open,high,low,close,Quant
Timestamp,,,,,
2019-11-26 09:03:00,4248.0,4248.5,4245.5,4246.5,7151
2019-11-26 09:04:00,4246.5,4251.5,4246.0,4248.5,3181
2019-11-26 09:05:00,4249.0,4250.5,4243.5,4244.0,2595
2019-11-26 09:06:00,4244.0,4247.0,4243.0,4245.5,1842
2019-11-26 09:07:00,4245.0,4249.0,4245.0,4249.0,926
2019-11-26 09:08:00,4248.5,4251.0,4247.0,4250.5,1561
2019-11-26 09:09:00,4250.5,4251.0,4248.5,4251.0,858
2019-11-26 09:10:00,4251.0,4253.5,4247.0,4247.5,3906
2019-11-26 09:11:00,4247.5,4249.0,4245.5,4249.0,2089
